In [1]:
import os
from dotenv import load_dotenv
HOME = os.getenv('HOME')
load_dotenv(f'{HOME}/.zshrc')

import sys
sys.path.insert(0, '../scripts')
import compare_methods as cm

import numpy as np
from sage.rings.polynomial.multi_polynomial_ideal import MPolynomialIdeal
from sage.misc.sage_eval import sage_eval
from sage.rings.rational_field import QQ

Python-dotenv could not parse statement starting at line 69
Python-dotenv could not parse statement starting at line 105
Python-dotenv could not parse statement starting at line 106
Python-dotenv could not parse statement starting at line 115
Python-dotenv could not parse statement starting at line 116
Python-dotenv could not parse statement starting at line 117
Python-dotenv could not parse statement starting at line 119
Python-dotenv could not parse statement starting at line 120
Python-dotenv could not parse statement starting at line 121
Python-dotenv could not parse statement starting at line 122


In [167]:
import json
from pprint import pprint
count = 0
with open('../data/h11_2.json', 'r') as fh:
    for line in fh:
        doc = json.loads(line)
        doc_ids = {k: v for k, v in doc.items() if k in ['H11', 'POLYID', 'GEOMN', 'TRIANGN', 'INVOLN']}
        if doc['OPLANES']['prev'] != doc['OPLANES']['curr']:
            print(doc['VOLFORMPARITY'])
            count += 1
            doc_ids.update({'OPLANES': doc['OPLANES']['prev']})
            pprint(doc_ids)
            doc_ids.update({'OPLANES': doc['OPLANES']['curr']})
            pprint(doc_ids)
            print('')

{'prev': 1, 'curr': 1}
{'GEOMN': 1,
 'H11': 2,
 'INVOLN': 1,
 'OPLANES': [{'ODIM': 5, 'OIDEAL': ['x0*x4 - x2*x5', '-x1*x2 + x0*x3']}],
 'POLYID': 10,
 'TRIANGN': 1}
{'GEOMN': 1,
 'H11': 2,
 'INVOLN': 1,
 'OPLANES': [{'ODIM': 5,
              'OIDEAL': ['x1*x4 - x3*x5', 'x0*x4 - x2*x5', '-x1*x2 + x0*x3']}],
 'POLYID': 10,
 'TRIANGN': 1}

{'prev': 1, 'curr': 1}
{'GEOMN': 1,
 'H11': 2,
 'INVOLN': 5,
 'OPLANES': [{'ODIM': 5, 'OIDEAL': ['x0*x3 - x4*x5', 'x0*x2 - x1*x4']}],
 'POLYID': 6,
 'TRIANGN': 1}
{'GEOMN': 1,
 'H11': 2,
 'INVOLN': 5,
 'OPLANES': [{'ODIM': 5,
              'OIDEAL': ['x1*x3 - x2*x5', 'x0*x3 - x4*x5', 'x0*x2 - x1*x4']}],
 'POLYID': 6,
 'TRIANGN': 1}

{'prev': 1.0, 'curr': 1}
{'GEOMN': 1,
 'H11': 2,
 'INVOLN': 2,
 'OPLANES': [{'ODIM': 5, 'OIDEAL': ['x0*x3 - x2*x5', '-x1*x2 + x0*x4']}],
 'POLYID': 6,
 'TRIANGN': 1}
{'GEOMN': 1,
 'H11': 2,
 'INVOLN': 2,
 'OPLANES': [{'ODIM': 5,
              'OIDEAL': ['x1*x2*x3 + x0*x3*x4 + x2*x4*x5',
                         'x0*x1*x3 + x

In [165]:
count

112

In [13]:
result = cm.sample_format_inputs({'H11': 3, 'VOLFORMPARITY': -1})

In [121]:
import numpy as np
invol = result['INVOL']
rescws = result['RESCWS']
X = np.array(R.gens())

In [129]:
invol

Permutation(6)(0, 3)(2, 5)

In [128]:
rescws

array([[0, 0, 1],
       [0, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [2, 2, 2]])

In [130]:
np.array(invol(rescws))

array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 1, 0],
       [1, 0, 0],
       [2, 2, 2]])

In [116]:
from itertools import combinations, permutations
from sympy.combinatorics.permutations import Permutation

In [144]:
a = np.empty(0)
np.append(a, [5])

array([5.])

In [141]:
a

array([4])

In [148]:
def compute_volformparity(rescws, invol):
    k, m = rescws.shape
    n = k - m
    vform = np.empty(0)
    vform_invol = np.empty(0)
    for c in combinations(range(k), n):
        term = 0
        term_invol = 0
        nc = tuple(set(range(k)) - set(c))
        for p in permutations(nc, k - n):
            p_c = Permutation(p + c)
            term += p_c.signature() * rescws[p, range(k - n)].prod()
            term_invol += (invol * p_c).signature() * rescws[invol.array_form][p, range(k - n)].prod()
        vform = np.append(vform, [term])
        vform_invol = np.append(vform_invol, [term_invol])
    if (vform == vform_invol).all():
        return 1
    elif (vform == -vform_invol).all():
        return -1
    else:
        return 0

In [151]:
vform == -vform_invol

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True])

In [117]:
k, n = result['DRESVERTS'].shape
R = cm.build_ring(k)
P_monoms = cm.compute_monomials(R, result['NVERTS'], result['DRESVERTS'])
srideal = cm.compute_srideal(R, result['TRIANG'])
fixed_loci = cm.compute_fixed_loci(R, P_monoms, srideal, result['INVOL'])
P_diff_ideal = R.ideal(*P_diff).radical()
P_monoms_ideal = R.ideal(*P_monoms).radical()
fixed_loci = []
for loc in P_diff_ideal.minimal_associated_primes():
    for s in srideal:
        if (P_monoms_ideal + loc).quotient(s).dimension() < 0:
            break
    else:
        fixed_loci.append(loc)

In [106]:
(R.ideal(*P_monoms).radical() + fixed_loci[0]).quotient(srideal[2])

Ideal (1) of Multivariate Polynomial Ring in x0, x1, x2, x3, x4, x5, x6 over Rational Field

In [15]:
X = np.array(R.gens())
X_sub = dict(zip(X, X[result['INVOL'].array_form]))
P_diff = [x - x.subs(X_sub) for x in P_monoms]
hysurf_diff_ideal = R.ideal(*P_diff).radical()
srideal_intersection = MPolynomialIdeal.intersection(*srideal).radical()
hysurf_diff_ideal = hysurf_diff_ideal.quotient(srideal_intersection)
fixed_loci1 = hysurf_diff_ideal.minimal_associated_primes()

In [108]:
result['OPLANES']

[{'OIDEAL': ['x1*x3-x4*x6'], 'ODIM': 7}]

In [113]:
fixed_loci

[Ideal (x4, x1) of Multivariate Polynomial Ring in x0, x1, x2, x3, x4, x5, x6 over Rational Field,
 Ideal (x0*x2 - x3*x5) of Multivariate Polynomial Ring in x0, x1, x2, x3, x4, x5, x6 over Rational Field]

In [32]:
srideal_intersection

Ideal (x3*x5*x6, x2*x5*x6, x0*x3*x6, x0*x2*x6, x3*x4*x5, x2*x4*x5, x1*x3*x5, x1*x2*x5, x0*x3*x4, x0*x2*x4, x0*x1*x3, x0*x1*x2) of Multivariate Polynomial Ring in x0, x1, x2, x3, x4, x5, x6 over Rational Field

In [51]:
a = (R.ideal(*P_monoms) + srideal_intersection).radical()
a

Ideal (x6, x3*x4*x5, x2*x4*x5, x1*x3*x5, x1*x2*x5, x0*x3*x4, x0*x2*x4, x0*x1*x3, x0*x1*x2) of Multivariate Polynomial Ring in x0, x1, x2, x3, x4, x5, x6 over Rational Field

In [98]:
(R.ideal(*P_monoms) + fixed_loci[0] + srideal_intersection).radical()

Ideal (x6, x4, x1) of Multivariate Polynomial Ring in x0, x1, x2, x3, x4, x5, x6 over Rational Field

In [81]:
srideal

[Ideal (x0, x5) of Multivariate Polynomial Ring in x0, x1, x2, x3, x4, x5, x6 over Rational Field,
 Ideal (x2, x3) of Multivariate Polynomial Ring in x0, x1, x2, x3, x4, x5, x6 over Rational Field,
 Ideal (x1, x4, x6) of Multivariate Polynomial Ring in x0, x1, x2, x3, x4, x5, x6 over Rational Field]

In [67]:
(R.ideal(*P_monoms) + fixed_loci[0]).radical().quotient(srideal[2])

Ideal (1) of Multivariate Polynomial Ring in x0, x1, x2, x3, x4, x5, x6 over Rational Field

In [59]:
srideal_intersection

Ideal (x3*x5*x6, x2*x5*x6, x0*x3*x6, x0*x2*x6, x3*x4*x5, x2*x4*x5, x1*x3*x5, x1*x2*x5, x0*x3*x4, x0*x2*x4, x0*x1*x3, x0*x1*x2) of Multivariate Polynomial Ring in x0, x1, x2, x3, x4, x5, x6 over Rational Field

In [36]:
(a + R.ideal(*P_diff)).minimal_associated_primes()

[Ideal (x6, x5, x0) of Multivariate Polynomial Ring in x0, x1, x2, x3, x4, x5, x6 over Rational Field,
 Ideal (x6, x4, x1) of Multivariate Polynomial Ring in x0, x1, x2, x3, x4, x5, x6 over Rational Field,
 Ideal (x6, x3, x2) of Multivariate Polynomial Ring in x0, x1, x2, x3, x4, x5, x6 over Rational Field]

In [21]:
[x.reduce(srideal_intersection) for x in P_monoms]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 x6^2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [66]:
R.ideal(sum(P_monoms), *P_diff).radical().quotient(srideal_intersection).minimal_associated_primes()

[Ideal (x3^10*x4^5 - x2*x3^8*x4^4*x5 + x2^2*x3^6*x4^3*x5^2 - x2^3*x3^4*x4^2*x5^3 + x2^4*x3^2*x4*x5^4 + x2^5*x5^5, x1*x3^9*x4^4 + x3^8*x4^4*x5 - x2*x3^6*x4^3*x5^2 + x2^2*x3^4*x4^2*x5^3 - x2^3*x3^2*x4*x5^4 - x2^4*x5^5, x1*x2 + x3*x4, x1^2*x3^8*x4^3 + x1*x3^7*x4^3*x5 + x3^6*x4^3*x5^2 - x2*x3^4*x4^2*x5^3 + x2^2*x3^2*x4*x5^4 + x2^3*x5^5, x1^3*x3^7*x4^2 + x1^2*x3^6*x4^2*x5 + x1*x3^5*x4^2*x5^2 + x3^4*x4^2*x5^3 - x2*x3^2*x4*x5^4 - x2^2*x5^5, x1^4*x3^6*x4 + x1^3*x3^5*x4*x5 + x1^2*x3^4*x4*x5^2 + x1*x3^3*x4*x5^3 + x3^2*x4*x5^4 + x2*x5^5, x1^5*x3^5 + x1^4*x3^4*x5 + x1^3*x3^3*x5^2 + x1^2*x3^2*x5^3 + x1*x3*x5^4 - x5^5, x6, x0) of Multivariate Polynomial Ring in x0, x1, x2, x3, x4, x5, x6 over Rational Field,
 Ideal (x1*x2 - x3*x4, x0^10*x3^6*x4^12 + 2*x0^9*x1*x3^6*x4^11 + 3*x0^8*x1^2*x3^6*x4^10 + 4*x0^7*x1^3*x3^6*x4^9 + x0^8*x1*x3^5*x4^10*x5 + 5*x0^6*x1^4*x3^6*x4^8 + 2*x0^7*x1^2*x3^5*x4^9*x5 + 6*x0^5*x1^5*x3^6*x4^7 + 3*x0^6*x1^3*x3^5*x4^8*x5 + 7*x0^4*x1^6*x3^6*x4^6 + 4*x0^5*x1^4*x3^5*x4^7*x5 + x0^6*

In [53]:
hysurf_diff_ideal.radical().change_ring(RR)

Ideal (0, 0, x1bar^2*x2bar*x6bar, 0, x1bar^3*x2bar^2) of Quotient of Multivariate Polynomial Ring in x0, x1, x2, x3, x4, x5, x6 over Rational Field by the ideal (x2*x5*x6, x3*x4*x6, x2*x3*x6, x0*x2*x6, x3*x4*x5, x2*x4*x5, x1*x2*x5, x1*x3*x4, x0*x3*x4, x0*x2*x4, x1*x2*x3, x0*x1*x2)

In [36]:
a = R.ideal(*[x.reduce(srideal_intersection + hysurf_diff_ideal) for x in P_monoms]).radical()

In [37]:
(a + hysurf_diff_ideal).radical().quotient(srideal_intersection).minimal_associated_primes()

[Ideal (x1*x2 - x3*x4, x6) of Multivariate Polynomial Ring in x0, x1, x2, x3, x4, x5, x6 over Rational Field,
 Ideal (x1*x2 + x3*x4, x6, x0) of Multivariate Polynomial Ring in x0, x1, x2, x3, x4, x5, x6 over Rational Field]

In [20]:
(fixed_loci[1] + R.ideal(*P_monoms)).radical()

Ideal (x6, x0, x3*x4, x1*x2, x2*x4*x5) of Multivariate Polynomial Ring in x0, x1, x2, x3, x4, x5, x6 over Rational Field

In [8]:
a = MPolynomialIdeal.intersection(*fixed_loci).radical()

In [140]:
(a + R.ideal(*P_monoms)).radical()#.quotient(srideal_intersection)

Ideal (x6, x3*x4*x5, x2*x4*x5, x1*x3*x5, x1*x2*x5, x0*x3*x4, x0*x2*x4, x0*x1*x3, x0*x1*x2) of Multivariate Polynomial Ring in x0, x1, x2, x3, x4, x5, x6 over Rational Field

In [146]:
fixed_loci

[Ideal (x5, x0) of Multivariate Polynomial Ring in x0, x1, x2, x3, x4, x5, x6 over Rational Field,
 Ideal (x1*x2 - x3*x4) of Multivariate Polynomial Ring in x0, x1, x2, x3, x4, x5, x6 over Rational Field]

In [147]:
fixed_loci1

[Ideal (x5, x0) of Multivariate Polynomial Ring in x0, x1, x2, x3, x4, x5, x6 over Rational Field,
 Ideal (x1*x2 - x3*x4) of Multivariate Polynomial Ring in x0, x1, x2, x3, x4, x5, x6 over Rational Field]

In [143]:
set(x.reduce(fixed_loci[0]) for x in P_monoms) - set([0])

{x6^2}

In [145]:
set(x.reduce(fixed_loci[2]) for x in P_monoms) - set([0])

IndexError: list index out of range

In [170]:
(fixed_loci[0] + R.ideal(*P_monoms) + srideal[0]).radical()#.quotient(srideal_intersection)

Ideal (x6, x5, x4, x1, x0) of Multivariate Polynomial Ring in x0, x1, x2, x3, x4, x5, x6 over Rational Field

In [162]:
srideal

[Ideal (x1, x4) of Multivariate Polynomial Ring in x0, x1, x2, x3, x4, x5, x6 over Rational Field,
 Ideal (x2, x3) of Multivariate Polynomial Ring in x0, x1, x2, x3, x4, x5, x6 over Rational Field,
 Ideal (x0, x5, x6) of Multivariate Polynomial Ring in x0, x1, x2, x3, x4, x5, x6 over Rational Field]

In [175]:
P_monoms[4], P_monoms[4].reduce(hysurf_diff_ideal)

(x0^4*x2^4*x4^4, x0^4*x2^4*x4^4)

In [172]:
P_monoms[0]

x0^4*x3^4*x4^4

In [197]:
(fixed_loci[2] + R.ideal(*[x + x.subs(X_sub) for x in P_monoms])).radical()

IndexError: list index out of range

In [194]:
R.ideal(*[x + x.subs(X_sub) for x in P_monoms)).radical()

Ideal (x6, x3*x4*x5, x2*x4*x5, x1*x3*x5, x1*x2*x5, x0*x3*x4, x0*x2*x4, x0*x1*x3, x0*x1*x2) of Multivariate Polynomial Ring in x0, x1, x2, x3, x4, x5, x6 over Rational Field